In [ ]:
#Modelo RANDOM FOREST para detección de fraudes

In [ ]:
# Librerías
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import joblib


# Cambiar directorio a 'outputs' para leer los CSV
os.chdir("C:/repo personal/PYTHON/deteccion de fraudes/outputs")

# Subir un nivel desde 'modelos/' a la raíz 'deteccion de fraudes/'
sys.path.append(os.path.abspath('..'))

# Importar la función ajustar_umbral desde utils.py
from scripts.utils import ajustar_umbral

In [ ]:
# Cargar datos
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv").squeeze()
y_test = pd.read_csv("y_test.csv").squeeze()

print(f"Tamaño de X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
# Aplicar SMOTE para balancear los datos de entrenamiento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("Datos balanceados con SMOTE:")
print(f"Tamaño original: {X_train.shape}, {y_train.shape}")
print(f"Tamaño balanceado: {X_train_balanced.shape}, {y_train_balanced.shape}")

In [ ]:
# Definir el modelo base
rf = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
    n_jobs = -1,
    verbose = 1)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=15,
    scoring='average_precision',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Ajustar el modelo
random_search.fit(X_train_balanced, y_train_balanced)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", random_search.best_params_)

In [ ]:
# Guardar el objeto completo
joblib.dump(random_search, '../outputs/random_search_rf.pkl')

# Para usar después y no correr el modelo completo de nuevo:
random_search = joblib.load('../outputs/random_search_rf.pkl')



In [ ]:
# Entrenar el modelo con los mejores hiperparámetros
best_rf = random_search.best_estimator_
best_rf.fit(X_train_balanced, y_train_balanced)

# Predicciones en el conjunto de prueba
y_pred = best_rf.predict(X_test)
y_prob = best_rf.predict_proba(X_test)[:, 1]

In [ ]:
# Ajustar el umbral de predicción
thresholds = np.arange(0.1, 1.0, 0.1)
resultados = ajustar_umbral(y_test, y_prob, thresholds)

# Convertir resultados a un DataFrame para visualización
resultados_df = pd.DataFrame(resultados, columns=["Threshold", "Precision", "Recall", "F1-Score"])

# Mostrar resultados
print(resultados_df)



In [ ]:
# Visualizar métricas en función del umbral
plt.figure(figsize=(10, 6))
plt.plot(resultados_df["Threshold"], resultados_df["Precision"], label="Precision", marker="o")
plt.plot(resultados_df["Threshold"], resultados_df["Recall"], label="Recall", marker="o")
plt.plot(resultados_df["Threshold"], resultados_df["F1-Score"], label="F1-Score", marker="o")
plt.title("Métricas en función del umbral")
plt.xlabel("Umbral")
plt.ylabel("Valor de la métrica")
plt.legend()
plt.grid()
plt.show()

# Seleccionar el mejor umbral basado en F1-Score
mejor_umbral = resultados_df.loc[resultados_df["F1-Score"].idxmax(), "Threshold"]
print(f"Mejor umbral basado en F1-Score: {mejor_umbral:.2f}")

# Generar predicciones finales con el mejor umbral
y_pred_final = (y_prob >= mejor_umbral).astype(int)

Dados estos resultados se podría tomar una decisión: mayor recall o mayor precision. En umbral 0.6 tenemos efectivamente el mayor F1, hay mayor precision pero menor recall, si queremos mejorar la detección de fraudes sacrificando un mayor número de falsos positivos entonces escoger 0.4 de umbral sería ideal.

In [ ]:
# Matriz de confusión con el mejor umbral
conf_matrix = confusion_matrix(y_test, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Clase 0", "Clase 1"], yticklabels=["Clase 0", "Clase 1"])
plt.title(f"Matriz de Confusión (Umbral = {mejor_umbral:.2f})")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

In [ ]:
# Reporte de clasificación con el mejor umbral
print("Reporte de Clasificación con el mejor umbral:")
print(classification_report(y_test, y_pred_final, digits=4))

In [ ]:
# Calcular Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.4f}")

In [ ]:
# Visualizar curva Precision-Recall
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f"PR-AUC = {pr_auc:.4f}", color="blue")
plt.title("Curva Precision-Recall")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.grid()
plt.show()